In [2]:
from google.colab import drive
from pathlib import Path
from PIL import Image
import cv2
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os

drive.mount('/content/drive')

!unzip /content/drive/MyDrive/IMSAI/image.zip -d /content/sample_data/

class CustomDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.image_files = list(self.image_dir.glob('*.png'))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (64, 64))
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        label = 0 if 'o' in img_path.stem else 1
        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = CustomDataset('/content/sample_data/image', transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Mounted at /content/drive
Archive:  /content/drive/MyDrive/IMSAI/image.zip
  inflating: /content/sample_data/image/o_1.png  
  inflating: /content/sample_data/image/o_10.png  
  inflating: /content/sample_data/image/o_100.png  
  inflating: /content/sample_data/image/o_101.jpg  
  inflating: /content/sample_data/image/o_102.jpg  
  inflating: /content/sample_data/image/o_103.jpg  
  inflating: /content/sample_data/image/o_104.jpg  
  inflating: /content/sample_data/image/o_105.jpg  
  inflating: /content/sample_data/image/o_106.jpg  
  inflating: /content/sample_data/image/o_107.jpg  
  inflating: /content/sample_data/image/o_108.jpg  
  inflating: /content/sample_data/image/o_109.jpg  
  inflating: /content/sample_data/image/o_11.png  
  inflating: /content/sample_data/image/o_110.jpg  
  inflating: /content/sample_data/image/o_111.jpg  
  inflating: /content/sample_data/image/o_112.jpg  
  inflating: /content/sample_data/image/o_113.jpg  
  inflating: /content/sample_data/image/o_114

In [5]:
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 8 * 8)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test dataset: {accuracy:.2f}%')


Epoch [1/25], Loss: 2.2008
Epoch [2/25], Loss: 0.7481
Epoch [3/25], Loss: 0.4700
Epoch [4/25], Loss: 0.3016
Epoch [5/25], Loss: 0.2451
Epoch [6/25], Loss: 0.1800
Epoch [7/25], Loss: 0.1314
Epoch [8/25], Loss: 0.0996
Epoch [9/25], Loss: 0.0410
Epoch [10/25], Loss: 0.0435
Epoch [11/25], Loss: 0.0378
Epoch [12/25], Loss: 0.0211
Epoch [13/25], Loss: 0.0196
Epoch [14/25], Loss: 0.0101
Epoch [15/25], Loss: 0.0093
Epoch [16/25], Loss: 0.0118
Epoch [17/25], Loss: 0.0109
Epoch [18/25], Loss: 0.0070
Epoch [19/25], Loss: 0.0057
Epoch [20/25], Loss: 0.0101
Epoch [21/25], Loss: 0.0074
Epoch [22/25], Loss: 0.0046
Epoch [23/25], Loss: 0.0054
Epoch [24/25], Loss: 0.0017
Epoch [25/25], Loss: 0.0173
Accuracy of the model on the test dataset: 97.50%
